In [29]:
import os
import requests
import numpy as np
import pandas as pd
import json
import csv
import psycopg2 
from io import StringIO
from datetime import datetime, timedelta
import requests
from dotenv import load_dotenv

from sqlalchemy import create_engine, text, DateTime
import pandera as pa
from pandera.errors import SchemaError
pd.set_option('display.max_rows', 500)
pd.options.display.max_columns = None

import write_to_db
import importlib
importlib.reload(write_to_db)

<module 'write_to_db' from '/app/jupyter_notebook/write_to_db.py'>

In [2]:
def convert_date_iso(dt):
   return dt.strftime('%Y-%m-%dT%H:%M:%S.%f%z')

def get_datetime(tz=None):
    if tz == None:
        datetime_now = datetime.now()
        datetime_str = datetime_now.strftime("%Y-%m-%dT%H:%M:%S.%f%z")
        return {"dt": datetime_now, "dt_str": datetime_str}

In [3]:
def get_engine():
    # load_dotenv(dotenv_path="./config/dev.env")
    DATABASE_URI = os.getenv("DATABASE_URI")
    engine = create_engine(DATABASE_URI)
    return engine


def query_bbdt_db(query):
    engine = get_engine()
    connection = engine.connect()
    df = pd.read_sql(text(query), connection)  # sql alchemy
    connection.close()  # not sure if I need this
    engine.dispose()  # not sure if I need this
    df = df.loc[:, ~df.columns.duplicated()].copy()
    return df


def query_record_db(query):
    engine = get_engine()
    connection = engine.connect()
    connection.execute(text(query))
    connection.close()  # not sure if I need this
    engine.dispose()  # not sure if I need this

In [15]:
query = f"select * from users ORDER BY update_datetime DESC LIMIT 1"
return_df = query_bbdt_db(query)
return_df

,username,create_datetime,update_datetime,metric,height,weight,gender,birth_date_datetime,bodyfat_calc,bodyfat_override_bool,bodyfat_override,bodyfat,lean_body_mass_calc,lean_body_mass_calc_category_id,activity_level_id,baseline_calorie_calc,baseline_calorie_override_bool,baseline_calorie_override,baseline_calorie,goal_id,goal_date_datetime,id,macro_adjustment_id
0,nikeshakya51@gmail.com,2023-12-28 14:39:51.781706,2023-12-28 14:40:02.337350,True,161,60.0,1,1999-12-27 18:15:00,12.0,False,12.0,12.0,116,100,2,1735,False,1735,1735,2,None,541,None


In [45]:
datetime_create = get_datetime()['dt']
insert_dict = {'datetime_create': datetime_create, 'username': 'test3', 'user_msg': 'test2', 'ai_msg': 'test2'}
# create dataframe from dictionary
df = pd.DataFrame([insert_dict])
#for col in df.columns:
#    if df[col].dtype in ["datetime64[ns]", "datetime64[ns, UTC]"]:
#        df[col] = df[col].dt.strftime("%Y-%m-%dT%H:%M:%S.%f%z")

In [43]:
engine = get_engine()
table_name = "ai_chat"
dtypes = {'datetime_create': DateTime}
df.to_sql(table_name, engine, if_exists="append", index=False, dtype=dtypes)

1

In [46]:
query = f"select * from ai_chat order by datetime_create DESC LIMIT 10"
return_df = query_bbdt_db(query)
return_df

,index,datetime_create,username,user_msg,ai_msg
0,NaN,2023-12-29 20:03:49.448729,dev@dev.com,am i a male or female,"Based on the information provided, you are a m..."
1,NaN,2023-12-29 19:56:53.221872,test3,test2,test2
2,0.0,2023-12-29 19:54:20.809182,test2,test2,test2
3,NaN,2023-12-29 19:54:20.809182,test2,test2,test2
4,0.0,2023-12-29 19:39:56.651756,test,test,test


In [32]:
query = f"select *  from information_schema.columns where table_name='ai_chat'"
return_df = query_bbdt_db(query)
return_df

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,numeric_precision,numeric_precision_radix,numeric_scale,datetime_precision,interval_type,interval_precision,character_set_catalog,character_set_schema,character_set_name,collation_catalog,collation_schema,collation_name,domain_catalog,domain_schema,domain_name,udt_catalog,udt_schema,udt_name,scope_catalog,scope_schema,scope_name,maximum_cardinality,dtd_identifier,is_self_referencing,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,postgres,public,ai_chat,index,1,None,YES,bigint,None,NaN,64.0,2.0,0.0,NaN,None,None,None,None,None,None,None,None,None,None,None,postgres,pg_catalog,int8,None,None,None,None,1,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
1,postgres,public,ai_chat,datetime_create,2,None,YES,timestamp without time zone,None,NaN,NaN,NaN,NaN,6.0,None,None,None,None,None,None,None,None,None,None,None,postgres,pg_catalog,timestamp,None,None,None,None,2,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
2,postgres,public,ai_chat,username,3,None,YES,text,None,1.073742e+09,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,postgres,pg_catalog,text,None,None,None,None,3,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
3,postgres,public,ai_chat,user_msg,4,None,YES,text,None,1.073742e+09,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,postgres,pg_catalog,text,None,None,None,None,4,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
4,postgres,public,ai_chat,ai_msg,5,None,YES,text,None,1.073742e+09,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,postgres,pg_catalog,text,None,None,None,None,5,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
